# Test Vectorize Module - Từng Bước
Notebook này chạy từng bước của quá trình vector hóa để debug

In [ ]:
import cv2
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import yaml
import sys
sys.path.append('.')

from pp.pp_ import cleaning_img
from predict_segment.pred.predict_segment import predict_segment
from vectorize.raw_img_process import rawFloorplanExtractor
from vectorize.clean_img_process import cleanFloorplanExtractor
from vectorize.merge.wall_check import wall_line_check
from vectorize.graph_process.build_graph import build_labeled_graph, split_edges_by_nodes, zip_thick_walls, simplify_graph_topology, orthogonalize_graph
from vectorize.graph_process.pulp_ import smart_snap_optimization
from vectorize.utils import plot_graph, to_json

## 1. Load Config và Ảnh

In [ ]:
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

image_path = config['main']['img_path']
loadmodel = config['main']['model_path']

print(f"Ảnh đầu vào: {image_path}")
print(f"Model: {loadmodel}")

## 2. Segment và Clean

In [ ]:
print('Đang phân đoạn ảnh...')
room_path, boundary_path = predict_segment(image_path, loadmodel, postprocess=True)
print(f"Room: {room_path}")
print(f"Boundary: {boundary_path}")

print('\nĐang làm sạch...')
clean_path = cleaning_img(boundary_path, room_path)
print(f"Clean: {clean_path}")

## 3. Trích xuất Vectors từ Raw và Clean

In [ ]:
# Load ảnh
raw = cv2.imread(image_path)
clean = cv2.imread(clean_path)

print(f'Raw shape: {raw.shape}')

# Trích xuất tường từ raw
raw_extractor = rawFloorplanExtractor()
binary, contour_img = raw_extractor.get_raw_contours(raw)
skeleton_image, vectors = raw_extractor.skeleton_to_vectors(binary)

print(f"Số lượng wall vectors: {len(vectors)}")

# Trích xuất cửa sổ từ clean
clean_extractor = cleanFloorplanExtractor(
    target_width=raw.shape[1],
    target_height=raw.shape[0],
    tolerance=45
)
window_mask, wall_mask = clean_extractor.process_img(clean)
win = clean_extractor.get_openings_skel(window_mask, kernel=1, iters=1)

print(f"Số lượng window vectors: {len(win)}")

## 4. Kiểm tra độ dài cửa sổ

In [ ]:
import math

window_lengths = []
for poly in win:
    for i in range(len(poly) - 1):
        p1 = poly[i]
        p2 = poly[i+1]
        length = math.hypot(p2[0]-p1[0], p2[1]-p1[1])
        window_lengths.append(length)

print(f"Tổng số đoạn cửa sổ: {len(window_lengths)}")
print(f"Độ dài min: {min(window_lengths):.2f}")
print(f"Độ dài max: {max(window_lengths):.2f}")
print(f"Độ dài trung bình: {np.mean(window_lengths):.2f}")
print(f"\nSố cửa sổ < 5 pixel: {sum(1 for l in window_lengths if l < 5)}")
print(f"Số cửa sổ < 10 pixel: {sum(1 for l in window_lengths if l < 10)}")
print(f"Số cửa sổ < 15 pixel: {sum(1 for l in window_lengths if l < 15)}")

## 5. Build Graph Ban Đầu

In [ ]:
G = build_labeled_graph(vectors, win)

print(f"Số nodes: {G.number_of_nodes()}")
print(f"Số edges: {G.number_of_edges()}")

# Đếm tường và cửa sổ
walls = [(u, v) for u, v, d in G.edges(data=True) if d.get('is_window') == False]
windows = [(u, v) for u, v, d in G.edges(data=True) if d.get('is_window') == True]

print(f"Số cạnh tường: {len(walls)}")
print(f"Số cạnh cửa sổ: {len(windows)}")

## 6. Bước 1: Split Edges by Nodes

In [ ]:
G1 = split_edges_by_nodes(G, tolerance=5)

print(f"\nSau split_edges_by_nodes:")
print(f"Số nodes: {G1.number_of_nodes()}")
print(f"Số edges: {G1.number_of_edges()}")

walls = [(u, v) for u, v, d in G1.edges(data=True) if d.get('is_window') == False]
windows = [(u, v) for u, v, d in G1.edges(data=True) if d.get('is_window') == True]

print(f"Số cạnh tường: {len(walls)}")
print(f"Số cạnh cửa sổ: {len(windows)}")

## 7. Bước 2: Smart Snap Optimization

In [ ]:
G2 = smart_snap_optimization(G1, snap_thresh=10)

print(f"\nSau smart_snap_optimization:")
print(f"Số nodes: {G2.number_of_nodes()}")
print(f"Số edges: {G2.number_of_edges()}")

walls = [(u, v) for u, v, d in G2.edges(data=True) if d.get('is_window') == False]
windows = [(u, v) for u, v, d in G2.edges(data=True) if d.get('is_window') == True]

print(f"Số cạnh tường: {len(walls)}")
print(f"Số cạnh cửa sổ: {len(windows)}")

## 8. Bước 3: Zip Thick Walls (QUAN TRỌNG - Kiểm tra xóa cửa sổ)

In [ ]:
# Kiểm tra trước khi zip
print("TRƯỚC KHI ZIP:")
windows_before = [(u, v, d) for u, v, d in G2.edges(data=True) if d.get('is_window') == True]
print(f"Số cửa sổ: {len(windows_before)}")
print(f"Độ dài cửa sổ nhỏ nhất: {min([d['weight'] for u, v, d in windows_before]):.2f}")

# Chạy zip
G3 = zip_thick_walls(G2, thickness_threshold=5)

# Kiểm tra sau khi zip
print(f"\nSAU KHI ZIP:")
print(f"Số nodes: {G3.number_of_nodes()}")
print(f"Số edges: {G3.number_of_edges()}")

walls = [(u, v) for u, v, d in G3.edges(data=True) if d.get('is_window') == False]
windows_after = [(u, v, d) for u, v, d in G3.edges(data=True) if d.get('is_window') == True]

print(f"Số cạnh tường: {len(walls)}")
print(f"Số cạnh cửa sổ: {len(windows_after)}")

print(f"\n⚠️ Số cửa sổ bị mất: {len(windows_before) - len(windows_after)}")

## 9. Bước 4: Remove Self Loops

In [ ]:
G3.remove_edges_from(nx.selfloop_edges(G3))

print(f"Sau remove self loops:")
print(f"Số nodes: {G3.number_of_nodes()}")
print(f"Số edges: {G3.number_of_edges()}")

## 10. Bước 5: Orthogonalize Graph

In [ ]:
G4 = orthogonalize_graph(G3, angle_threshold=45, iterations=30)

print(f"\nSau orthogonalize_graph:")
print(f"Số nodes: {G4.number_of_nodes()}")
print(f"Số edges: {G4.number_of_edges()}")

walls = [(u, v) for u, v, d in G4.edges(data=True) if d.get('is_window') == False]
windows = [(u, v) for u, v, d in G4.edges(data=True) if d.get('is_window') == True]

print(f"Số cạnh tường: {len(walls)}")
print(f"Số cạnh cửa sổ: {len(windows)}")

## 11. Bước 6: Simplify Graph Topology

In [ ]:
G5 = simplify_graph_topology(G4, collinear_threshold=0.2)

print(f"\nSau simplify_graph_topology:")
print(f"Số nodes: {G5.number_of_nodes()}")
print(f"Số edges: {G5.number_of_edges()}")

walls = [(u, v) for u, v, d in G5.edges(data=True) if d.get('is_window') == False]
windows = [(u, v) for u, v, d in G5.edges(data=True) if d.get('is_window') == True]

print(f"Số cạnh tường: {len(walls)}")
print(f"Số cạnh cửa sổ: {len(windows)}")

## 12. Visualize Kết Quả

In [ ]:
plt.figure(figsize=(15, 15))
pos = {n: n for n in G5.nodes()}

wall_edges = [(u, v) for u, v, d in G5.edges(data=True) if d.get('is_window') == False]
window_edges = [(u, v) for u, v, d in G5.edges(data=True) if d.get('is_window') == True]

nx.draw_networkx_edges(G5, pos, edgelist=wall_edges, edge_color='orange', width=3, label='Tường')
nx.draw_networkx_edges(G5, pos, edgelist=window_edges, edge_color='cyan', width=3, style='dashed', label='Cửa sổ')
nx.draw_networkx_nodes(G5, pos, node_size=30, node_color='red')

plt.gca().invert_yaxis()
plt.axis('equal')
plt.legend()
plt.title('Kết quả cuối cùng')
plt.show()

## 13. Xuất JSON

In [ ]:
import os

json_name = 'test_output.json'
to_json(G5.edges(data=True), file_name=json_name)
print(f'Đã lưu vào {json_name}')